### Function Data Sampling
Generate samples from a dataset generated by _process_line_level_data.py_.

In [12]:
import pandas as pd
import numpy as np
import math
import csv

In [4]:
df = pd.read_csv('./big-vul_dataset/processed_data.csv', skipinitialspace=True, low_memory = True)

C:\Users\Elias\AppData\Local\Temp/ipykernel_2604/4137211306.py:1: DtypeWarning: Columns (20,22,23,27,28,29,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./big-vul_dataset/processed_data.csv', skipinitialspace=True, low_memory = True)


In [5]:
len(df)

188636

In [6]:
# sorta weird NaN check (x == NaN if x != x)
df_with_vul = df.loc[df['flaw_line'] == df['flaw_line']]
len(df_with_vul)

# Also noteworthy:
# Why does this yield 8243 results while 10900 entries in the original dataset have vul = 1 ?

8243

In [7]:
# sorta weird NaN check (x == NaN if x != x)
df_without_vul = df.loc[df['flaw_line'] != df['flaw_line']]
len(df_without_vul)

180393

In [8]:
assert len(df) == len(df_with_vul) + len(df_without_vul), 'Some entries were lost during vul-filtering. Please check code!'

In [9]:
df_with_vul.tail(10)

,index,Access Gained,Attack Origin,Authentication Required,Availability,CVE ID,CVE Page,CWE ID,Complexity,Confidentiality,...,parentID,patch,project,project_after,project_before,target,vul_func_with_fix,processed_func,flaw_line,flaw_line_index
188616,188616,None,Remote,Not required,Complete,CVE-2016-1621,https://www.cvedetails.com/cve/CVE-2016-1621/,CWE-119,Low,Complete,...,NaN,"@@ -7,111 +7,271 @@\n\n * in the file PATENT...",Android,https://android.googlesource.com/platform/exte...,https://android.googlesource.com/platform/exte...,1,unsigned int subpel_avg_variance_ref(const uin...,unsigned int subpel_avg_variance_ref(const uin...,const uin...,"1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
188617,188617,None,Remote,Not required,Complete,CVE-2016-1621,https://www.cvedetails.com/cve/CVE-2016-1621/,CWE-119,Low,Complete,...,NaN,"@@ -7,111 +7,271 @@\n\n * in the file PATENT...",Android,https://android.googlesource.com/platform/exte...,https://android.googlesource.com/platform/exte...,1,static unsigned int subpel_variance_ref(const ...,static unsigned int subpel_variance_ref(const ...,int l2...,"1,2,3,4,5,8,9,10,11,12,13,14,15,16,17,18,21,22"
188618,188618,None,Remote,Not required,Complete,CVE-2016-1621,https://www.cvedetails.com/cve/CVE-2016-1621/,CWE-119,Low,Complete,...,NaN,"@@ -7,111 +7,271 @@\n\n * in the file PATENT...",Android,https://android.googlesource.com/platform/exte...,https://android.googlesource.com/platform/exte...,1,static unsigned int variance_ref(const uint8_t...,static unsigned int variance_ref(const uint8_t...,"int l2w, int ...","1,2,3,4,5,6,7,8,9,10,12,13"
188620,188620,None,Remote,Not required,Complete,CVE-2016-1621,https://www.cvedetails.com/cve/CVE-2016-1621/,CWE-119,Low,Complete,...,NaN,"@@ -11,28 +11,40 @@\n\n #include <string>\n \n...",Android,https://android.googlesource.com/platform/exte...,https://android.googlesource.com/platform/exte...,1,"string DecodeFile(const string& filename, int...","string DecodeFile(const string& filename, int ...",vpx_codec_dec_cfg_t cfg = {0};,4
188621,188621,None,Remote,Not required,Complete,CVE-2016-1621,https://www.cvedetails.com/cve/CVE-2016-1621/,CWE-119,Low,Complete,...,NaN,"@@ -11,28 +11,40 @@\n\n #include <string>\n \n...",Android,https://android.googlesource.com/platform/exte...,https://android.googlesource.com/platform/exte...,1,virtual void SetUp() {\n//flaw_line_below:\...,virtual void SetUp() {\n vp9_worker_init(&w...,vp9_worker_init(&worker_);,1
188622,188622,None,Remote,Not required,Complete,CVE-2016-1621,https://www.cvedetails.com/cve/CVE-2016-1621/,CWE-119,Low,Complete,...,NaN,"@@ -11,28 +11,40 @@\n\n #include <string>\n \n...",Android,https://android.googlesource.com/platform/exte...,https://android.googlesource.com/platform/exte...,1,virtual void TearDown() {\n//flaw_line_belo...,virtual void TearDown() {\n vp9_worker_end(...,vp9_worker_end(&worker_);,1
188630,188630,None,Remote,Not required,Complete,CVE-2016-0835,https://www.cvedetails.com/cve/CVE-2016-0835/,CWE-119,Low,Complete,...,NaN,"@@ -60,7 +60,7 @@\n\n *\n * Values Returned :...",Android,https://android.googlesource.com/platform/exte...,https://android.googlesource.com/platform/exte...,1,IMPEG2D_ERROR_CODES_T impeg2d_dec_p_b_slice(de...,IMPEG2D_ERROR_CODES_T impeg2d_dec_p_b_slice(de...,impeg2d_dec_pnb_mb_params(ps_dec);/~/ ...,"44,46"
188633,188633,None,Remote,Not required,None,CVE-2017-0402,https://www.cvedetails.com/cve/CVE-2017-0402/,CWE-200,Medium,Partial,...,NaN,"@@ -265,8 +265,12 @@\n\n case EQ_PARAM_BAN...",Android,https://android.googlesource.com/platform/hard...,https://android.googlesource.com/platform/hard...,1,int equalizer_get_parameter(effect_context_t *...,int equalizer_get_parameter(effect_context_t *...,if (param2 >= NUM_EQ_BANDS) {/~/ if (pa...,"78,89,90,100"
188634,188634,None,Remote,Not required,None,CVE-2018-9503,https://www.cvedetails.com/cve/CVE-2018-9503/,CWE-125,Low,Complete,...,NaN,"@@ -517,7 +517,16 @@\n\n return (RFC_EVENT...",Android,https://android.googlesourc

In [13]:
# Split all data into 90% training and 10% validation
from sklearn.model_selection import train_test_split
train_vul, val_vul = train_test_split(df_with_vul, test_size=0.1)
train_no_vul, val_no_vul = train_test_split(df_without_vul, test_size=0.1)

sample_df_all_train = pd.concat([train_vul, train_no_vul], ignore_index=True)
sample_df_all_train.to_csv('./big-vul_dataset/validation_split/train_all.csv', encoding='utf-8', quoting=csv.QUOTE_ALL)

sample_df_all_val = pd.concat([val_vul, val_no_vul], ignore_index=True)
sample_df_all_val.to_csv('./big-vul_dataset/validation_split/validation_all.csv', encoding='utf-8', quoting=csv.QUOTE_ALL)

In [ ]:
# Sample 10 percent of original dataset while keeping the original ratio of vul to non-vul 
sample_df_with_vul = df_with_vul.sample(frac=0.1)
sample_df_without_vul = df_without_vul.sample(frac=0.1)

sample_df_all = pd.concat([sample_df_with_vul, sample_df_without_vul], ignore_index=True)
sample_df_all.to_csv('./big-vul_dataset/sample_10p_original_ratio.csv', encoding='utf-8')

In [ ]:
# Create a balanced dataset between vul and non-vul by keeping all original vulnerable entires and 
# sampling the same number of non-vulnerable entries
sample_df_without_vul = df_without_vul.sample(len(df_with_vul))

sample_df_all = pd.concat([df_with_vul, sample_df_without_vul], ignore_index=True)
sample_df_all.to_csv('./big-vul_dataset/sample_all_balanced_ratio.csv', encoding='utf-8')

In [ ]:
# Create a balanced dataset between vul and non-vul by sampling 20% of vulnerable entires and 
# adding the same number of non-vulnerable
sample_df_with_vul = df_with_vul.sample(frac=0.2)
sample_df_without_vul = df_without_vul.sample(len(sample_df_with_vul))

sample_df_all = pd.concat([sample_df_with_vul, sample_df_without_vul], ignore_index=True)
sample_df_all.to_csv('./big-vul_dataset/sample_20p_balanced_ratio.csv', encoding='utf-8')